<a href="https://colab.research.google.com/github/shannn1/goodRAG/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/RAG
!pwd

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/MyDrive/RAG'
/content
/content


In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 22.8 MB/s eta 0:00:00


In [3]:
import faiss

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
import os
# from modeling_rag import RagTokenForGeneration
# from retrieval_rag import RagRetriever
# from tokenization_rag import RagTokenizer
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
from transformers import TrainingArguments, Trainer
from datasets import load_dataset

dataset = load_dataset("Shannnh/small-dataset-test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/414 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3 [00:00<?, ? examples/s]

In [6]:
dataset = dataset["train"]

In [7]:
import faiss
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np

In [9]:
embeddings = np.array(dataset['embeddings']).astype('float32')
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # 使用 L2 距离度量
index.add(embeddings)  # 添加嵌入到索引中
faiss.write_index(index, "faiss_index")

In [10]:
output_path = "./small_dataset_test"  # 保存路径
dataset.save_to_disk(output_path)

Saving the dataset (0/1 shards):   0%|          | 0/3 [00:00<?, ? examples/s]

In [11]:
model_name = "facebook/rag-sequence-nq"
tokenizer = RagTokenizer.from_pretrained(model_name)

retriever = RagRetriever.from_pretrained(
    model_name,
    index_name="custom",            # 使用自定义索引
    passages_path="small_dataset_test", # 知识库数据集的路径
    index_path="faiss_index",       # FAISS 索引文件的路径
    repo_type="dataset"             # 显式指定 repo_type
)

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

(…)_encoder_tokenizer/tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

question_encoder_tokenizer/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)ncoder_tokenizer/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


(…)enerator_tokenizer/tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

generator_tokenizer/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

generator_tokenizer/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)erator_tokenizer/special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may res

In [14]:
print(dataset.column_names)

['title', 'text', 'question', 'answer', 'embeddings']


In [13]:
model = RagTokenForGeneration.from_pretrained(model_name, retriever=retriever)

# 加载训练和验证数据集（替换为实际训练和验证数据集）
training_dataset = dataset  # 替换实际训练数据集
validation_dataset = dataset  # 替换实际验证数据集

# 配置训练参数
training_args = TrainingArguments(
    output_dir="./rag_finetune",
    evaluation_strategy="steps",
    eval_steps=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=5e-5,
    num_train_epochs=1,
    save_steps=1,
    logging_dir="./logs",
    logging_steps=1,
    overwrite_output_dir=True
)

# 定义 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)

# 开始训练
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-4183f727f4da>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [title, answer, text, question, embeddings]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.